In [1]:
%defaultDatasource jdbc:h2:mem:db

# Base de Dados

In [2]:
CREATE TABLE column_3C_weka (
    pelvic_incidence DOUBLE,
    pelvic_tilt DOUBLE,
    lumbar_lordosis_angle DOUBLE,
    sacral_slope DOUBLE,
    pelvic_radius DOUBLE,
    degree_spondylolisthesis DOUBLE,
    class VARCHAR(50)
) AS SELECT
    pelvic_incidence,
    pelvic_tilt,
    lumbar_lordosis_angle,
    sacral_slope,
    pelvic_radius,
    degree_spondylolisthesis,
    class
FROM CSVREAD('../data/column_3C_weka.csv');

ALTER TABLE column_3c_weka
ADD idPaciente INTEGER AUTO_INCREMENT;

In [3]:
SELECT * FROM column_3c_weka;

# Modelo Conceitual

### Antes:
![uml-etapa01](../img/uml-etapa01.png)

### Depois:
![uml-etapa02](../img/UML-etapa-02.png)

#### Motivo
* Modificação nos atributos da classe Paciente: agora ela possui somente o identificador do paciente chamado idPaciente, os atributos referentes à situação do mesmo (como Incidência Pélvica, Versão Pélvica, etc) foram transferidos para outras classes a fim de que se pudesse, caso surgisse uma nova característica (novo tipo de medição da Pelve, por exemplo) seria possível adicioná-la ao banco;  
* Seguindo a modificação do Paciente, foram adicionadas duas novas classes: Possui e Característica. Cada Paciente possui várias características (N:N), sendo que o valor do relacionamento entre o Paciente e sua Característica fica na classe intermediária Possui - por exemplo, João (Paciente) tem uma Incidência Pélvica (Característica) de valor 60° (Possui) e Maria (Paciente) tem uma Incidência Pélvica (Característica) de 70° (Possui);  
* Mudança no nome da classe Doença para Classificação e tipoDoença para classe, pois havia certos pacientes classificados como “Normal”, o que não caracterizava um problema.  


# Modelo Logico

![modelo-logico](../img/modelo-logico.png)

# Tabelas Relacionais

In [4]:
DROP TABLE IF EXISTS ClassificacaoTemSintoma;
DROP TABLE IF EXISTS PacientePossuiCaracteristica;
DROP TABLE IF EXISTS Caracteristica;
DROP TABLE IF EXISTS Sintoma;
DROP TABLE IF EXISTS Classificacao;
DROP TABLE IF EXISTS Paciente;

CREATE TABLE Caracteristica (
    nomeCaracteristica VARCHAR(50),
    PRIMARY KEY(nomeCaracteristica)
);

CREATE TABLE Sintoma (
    idSintoma INTEGER AUTO_INCREMENT,
    localizacaoCorpo VARCHAR(50),
    descricao VARCHAR(200),
    PRIMARY KEY(idSintoma)
);

CREATE TABLE Classificacao (
    classe VARCHAR(50),
    PRIMARY KEY(classe)
);

CREATE TABLE ClassificacaoTemSintoma (
    IdClassificacaoTemSintoma INTEGER AUTO_INCREMENT,
    classe VARCHAR(50),
    idSintoma INTEGER,
    FOREIGN KEY(classe)
        REFERENCES Classificacao(classe)
            ON DELETE CASCADE
            ON UPDATE CASCADE,
    FOREIGN KEY(idSintoma)
        REFERENCES Sintoma(idSintoma)
          ON DELETE CASCADE
          ON UPDATE CASCADE
);

CREATE TABLE Paciente (
    idPaciente INTEGER AUTO_INCREMENT,
    classe VARCHAR(50),
    PRIMARY KEY(idPaciente),
    FOREIGN KEY(classe)
        REFERENCES Classificacao(classe)
          ON DELETE CASCADE
          ON UPDATE CASCADE
);

CREATE TABLE PacientePossuiCaracteristica (
    idPacienteCaracteristica INTEGER AUTO_INCREMENT, 
    idPaciente INTEGER,
    nomeCaracteristica VARCHAR(50), 
    valor DOUBLE,
    PRIMARY KEY(idPacienteCaracteristica),
    FOREIGN KEY(idPaciente)
        REFERENCES Paciente(idPaciente)
          ON DELETE CASCADE
          ON UPDATE CASCADE,
    FOREIGN KEY(nomeCaracteristica)
        REFERENCES Caracteristica(nomeCaracteristica)
          ON DELETE CASCADE
          ON UPDATE CASCADE
);

# Inserção dos dados no modelo lógico

Inserção das classificações possíveis (Hérnia, Normal e Espondilolistese)

In [5]:
INSERT INTO Classificacao(classe)
SELECT DISTINCT class FROM column_3C_weka;

Inserção dos pacientes

In [6]:
INSERT INTO Paciente(idPaciente, classe)
SELECT idPaciente, class FROM column_3c_weka;

Inserção das características dos pacientes (Inclinação Sacral, Versão Pélvica, etc)

In [7]:
INSERT INTO Caracteristica (nomeCaracteristica) VALUES 
('Incidencia Pelvica'),
('Versao Pelvica'),
('Angulo Lordose Lombar'),
('Inclinacao Sacral'),
('Raio Pelvico'),
('Grau de Espondilolistese'); 

Inserção dos valores das características de cada paciente

In [8]:
INSERT INTO PacientePossuiCaracteristica (idPaciente, valor)
SELECT idPaciente, pelvic_incidence FROM column_3c_weka;

UPDATE PacientePossuiCaracteristica SET NomeCaracteristica = 'Incidencia Pelvica' 
WHERE idPacienteCaracteristica >= 1 AND idPacienteCaracteristica <= 310;

In [9]:
INSERT INTO PacientePossuiCaracteristica (idPaciente, valor)
SELECT idPaciente, pelvic_tilt FROM column_3c_weka;

UPDATE PacientePossuiCaracteristica SET NomeCaracteristica = 'Versao Pelvica' 
WHERE idPacienteCaracteristica >= 311 AND idPacienteCaracteristica <= 620;

In [10]:
INSERT INTO PacientePossuiCaracteristica (idPaciente, valor)
SELECT idPaciente, lumbar_lordosis_angle FROM column_3c_weka;

UPDATE PacientePossuiCaracteristica SET NomeCaracteristica = 'Angulo Lordose Lombar' 
WHERE idPacienteCaracteristica >= 621 AND idPacienteCaracteristica <= 930;

In [11]:
INSERT INTO PacientePossuiCaracteristica (idPaciente, valor)
SELECT idPaciente, sacral_slope FROM column_3c_weka;

UPDATE PacientePossuiCaracteristica SET NomeCaracteristica = 'Inclinacao Sacral' 
WHERE idPacienteCaracteristica >= 931 AND idPacienteCaracteristica <= 1240;

In [12]:
INSERT INTO PacientePossuiCaracteristica (idPaciente, valor)
SELECT idPaciente, pelvic_radius FROM column_3c_weka;

UPDATE PacientePossuiCaracteristica SET NomeCaracteristica = 'Raio Pelvico' 
WHERE idPacienteCaracteristica >= 1241 AND idPacienteCaracteristica <= 1550;

In [13]:
INSERT INTO PacientePossuiCaracteristica (idPaciente, valor)
SELECT idPaciente, degree_spondylolisthesis FROM column_3c_weka;

UPDATE PacientePossuiCaracteristica SET NomeCaracteristica = 'Grau de Espondilolistese' 
WHERE idPacienteCaracteristica >= 1551 AND idPacienteCaracteristica <= 1860;

Inserção dos sintomas possíveis das classificações (Hérnia e Espondilolistese)

In [14]:
INSERT INTO Sintoma (localizacaoCorpo, descricao) VALUES 
('Coluna Lombar','Dor na coluna lombar, parte inferior das costas'),
('Glúteos, coxas','Dor ciática'),
('Glúteos, braços, mãos','Dormência'),
('Corpo todo','Diminuição da força'),
('Pernas','Dor ao caminhar'),
('Pernas','Fraqueza'),
('Coluna, glúteos, perna, calcanhar','Dor intensa'),
('Nuca, pescoço','Dor'),
('Corpo todo','Dificuldade em se movimentar e abaixar'),
('Coluna', 'Sensação de Queimação');

In [15]:
INSERT INTO ClassificacaoTemSintoma (classe, idSintoma) VALUES
('Spondylolisthesis', 1),
('Spondylolisthesis', 2),
('Spondylolisthesis', 3),
('Spondylolisthesis', 4),
('Spondylolisthesis', 5),
('Hernia', 6),
('Hernia', 7),
('Hernia', 8),
('Hernia', 9),
('Hernia', 10);

In [16]:
SELECT * FROM Classificacao;
SELECT * FROM Sintoma;
SELECT * FROM ClassificacaoTemSintoma;
SELECT * FROM Paciente;
SELECT * FROM Caracteristica;
SELECT * FROM PacientePossuiCaracteristica;

# Querys Criativas

In [17]:
-- mostra doenca que a pessoa tem dado raio pelvico > 140
SELECT p.idpaciente, p.classe, ppc.valor
FROM PacientePossuiCaracteristica ppc, Paciente p
WHERE ppc.valor > 140 and ppc.nomecaracteristica = 'Raio Pelvico' and p.idpaciente = ppc.idpaciente

In [18]:
-- mostra doenca que a pessoa tem dado Angulo Lordose Lombar
SELECT p.idpaciente, p.classe, ppc.valor
FROM PacientePossuiCaracteristica ppc, Paciente p
WHERE ppc.valor > 80 and ppc.nomecaracteristica = 'Angulo Lordose Lombar' and p.idpaciente = ppc.idpaciente

In [19]:
-- conta a quantidade de ocorrencias de cada doenca nessa base de dados
SELECT p.classe, count(*) as ocorrencias
FROM Paciente p
GROUP BY p.classe;

In [20]:
-- permite ver os min e max de cada atributo dado a classificacao do paciente
SELECT p.classe, ppc.nomecaracteristica, min(ppc.valor), max(ppc.valor), avg(ppc.valor)
FROM Paciente p, PacientePossuiCaracteristica ppc
WHERE p.idpaciente = ppc.idpaciente and p.classe = 'Normal'
GROUP BY ppc.nomecaracteristica;

SELECT p.classe, ppc.nomecaracteristica, min(ppc.valor), max(ppc.valor), avg(ppc.valor)
FROM Paciente p, PacientePossuiCaracteristica ppc
WHERE p.idpaciente = ppc.idpaciente and p.classe = 'Spondylolisthesis'
GROUP BY ppc.nomecaracteristica;

SELECT p.classe, ppc.nomecaracteristica, min(ppc.valor), max(ppc.valor), avg(ppc.valor)
FROM Paciente p, PacientePossuiCaracteristica ppc
WHERE p.idpaciente = ppc.idpaciente and p.classe = 'Hernia'
GROUP BY ppc.nomecaracteristica;

-- todos
SELECT ppc.nomecaracteristica, min(ppc.valor), max(ppc.valor), avg(ppc.valor)
FROM Paciente p, PacientePossuiCaracteristica ppc
WHERE p.idpaciente = ppc.idpaciente
GROUP BY ppc.nomecaracteristica;

In [21]:
-- conta quantos pacientes estao acima da media
SELECT nomeCaracteristica, COUNT(*) Pacientes
FROM PacientePossuiCaracteristica ppc
WHERE nomeCaracteristica = 'Incidencia Pelvica'
AND Valor > (SELECT avg(pp.valor)
             FROM PacientePossuiCaracteristica pp
             WHERE nomeCaracteristica = 'Incidencia Pelvica'
             GROUP BY pp.nomecaracteristica)
GROUP BY nomeCaracteristica;

In [22]:
-- quantidade de pessoas que relataram dores em certas regioes do corpo
SELECT s.localizacaocorpo, count(*)
FROM ClassificacaoTemSintoma cts, Sintoma s, Paciente p
WHERE cts.idsintoma = s.idsintoma and p.classe = cts.classe
GROUP BY s.localizacaocorpo;

In [28]:
-- analise de pares
CREATE OR REPLACE VIEW EfeitoCruzado AS
SELECT DISTINCT ppc1.idPaciente Paciente, ppc1.nomeCaracteristica Atributo1, ppc2.nomeCaracteristica Atributo2
FROM PacientePossuiCaracteristica ppc1, PacientePossuiCaracteristica ppc2
WHERE ppc1.idPaciente = ppc2.idPaciente
AND ppc1.nomeCaracteristica <> ppc2.nomeCaracteristica
AND ppc1.nomeCaracteristica > ppc2.nomeCaracteristica
AND ppc1.valor > (SELECT avg(pp.valor)
                    FROM PacientePossuiCaracteristica pp
                    WHERE nomeCaracteristica = ppc1.nomeCaracteristica
                    GROUP BY pp.nomecaracteristica)
AND ppc2.valor > (SELECT avg(pp.valor)
                    FROM PacientePossuiCaracteristica pp
                    WHERE nomeCaracteristica = ppc2.nomeCaracteristica
                    GROUP BY pp.nomecaracteristica)
ORDER BY ppc1.idPaciente;

SELECT * FROM EfeitoCruzado;

# Salvar em CSV